# IBM DB2 Sales Pipeline Analytics Demo

This notebook demonstrates SQL queries on IBM sales pipeline data and generates visualizations.

In [1]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Set style
plt.style.use('default')
sns.set_palette("husl")

print("📚 Libraries loaded successfully")

ModuleNotFoundError: No module named 'pandas'

## 1. Create Demo Database and Tables

In [2]:
# Create demo database
conn = sqlite3.connect(':memory:')  # In-memory database

# Sample Pipeline data based on IBM MQT structure
pipeline_data = {
    'CALL_AMT': [100000, 0, 250000, 0, 500000, 150000, 0, 300000, 0, 750000],
    'WON_AMT': [0, 0, 0, 250000, 0, 0, 180000, 0, 400000, 0],
    'UPSIDE_AMT': [0, 150000, 0, 0, 0, 0, 0, 200000, 0, 0],
    'QUALIFY_PLUS_AMT': [100000, 150000, 250000, 250000, 500000, 150000, 180000, 500000, 400000, 750000],
    'OPPORTUNITY_VALUE': [120000, 180000, 300000, 250000, 600000, 200000, 180000, 500000, 400000, 800000],
    'PPV_AMT': [80000, 90000, 200000, 250000, 400000, 120000, 180000, 300000, 400000, 600000],
    'SALES_STAGE': ['Negotiate', 'Propose', 'Closing', 'Won', 'Qualify', 'Closing', 'Won', 'Propose', 'Won', 'Closing'],
    'GEOGRAPHY': ['Americas', 'Americas', 'EMEA', 'APAC', 'Americas', 'EMEA', 'APAC', 'Americas', 'EMEA', 'Americas'],
    'MARKET': ['US Federal', 'US Commercial', 'UK Market', 'Japan Market', 'Canada Market', 'Germany Market', 'Australia', 'US Federal', 'France', 'US Commercial'],
    'CLIENT_NAME': ['Federal Agency A', 'Corp B', 'UK Corp C', 'Japan Inc', 'Canada Ltd', 'German Co', 'Aussie Corp', 'Federal Agency B', 'French Co', 'Big Corp'],
    'DEAL_SIZE': ['large', 'medium', 'large', 'medium', 'large', 'medium', 'small', 'large', 'large', 'large'],
    'INDUSTRY': ['Government', 'Financial', 'Manufacturing', 'Technology', 'Healthcare', 'Automotive', 'Retail', 'Government', 'Energy', 'Financial'],
    'UT17_NAME': ['Cybersecurity', 'Hybrid Cloud & Data', 'Strategy & Transformation', 'AI/ML Ops', 'Application Operations', 'Business Applications', 'Data Fabric', 'Cybersecurity', 'Strategy & Transformation', 'Hybrid Cloud & Data'],
    'YEAR': [2025, 2025, 2025, 2025, 2025, 2025, 2025, 2025, 2025, 2025],
    'QUARTER': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    'WEEK': [3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
}

pipeline_df = pd.DataFrame(pipeline_data)
pipeline_df.to_sql('PROD_MQT_CONSULTING_PIPELINE', conn, if_exists='replace', index=False)

# Sample Budget data
budget_data = {
    'GEOGRAPHY': ['Americas', 'EMEA', 'APAC', 'Americas', 'EMEA', 'APAC'],
    'MARKET': ['US Federal', 'UK Market', 'Japan Market', 'US Commercial', 'Germany Market', 'Australia'],
    'REVENUE_BUDGET_AMT': [2000000, 1500000, 1200000, 1800000, 1000000, 800000],
    'YEAR': [2025, 2025, 2025, 2025, 2025, 2025],
    'QUARTER': [1, 1, 1, 1, 1, 1]
}

budget_df = pd.DataFrame(budget_data)
budget_df.to_sql('PROD_MQT_CONSULTING_BUDGET', conn, if_exists='replace', index=False)

print("✅ Demo tables created successfully")
print(f"Pipeline table: {len(pipeline_df)} rows")
print(f"Budget table: {len(budget_df)} rows")

NameError: name 'sqlite3' is not defined

## 2. Execute Sample Queries

### Query 1: What is the total value of deals currently in the pipeline?

In [3]:
query1 = """
SELECT 
    SUM(OPPORTUNITY_VALUE) / 1000000.0 AS TOTAL_PIPELINE_M,
    SUM(QUALIFY_PLUS_AMT) / 1000000.0 AS QUALIFIED_PIPELINE_M,
    SUM(PPV_AMT) / 1000000.0 AS PREDICTED_PIPELINE_M,
    COUNT(DISTINCT CLIENT_NAME) AS UNIQUE_CLIENTS,
    COUNT(*) AS TOTAL_OPPORTUNITIES
FROM 
    PROD_MQT_CONSULTING_PIPELINE
WHERE 
    SALES_STAGE NOT IN ('Won', 'Lost');
"""

df1 = pd.read_sql_query(query1, conn)
print("📊 TOTAL PIPELINE VALUE:")
print(f"Total Pipeline: ${df1['TOTAL_PIPELINE_M'].iloc[0]:.2f}M")
print(f"Qualified Pipeline: ${df1['QUALIFIED_PIPELINE_M'].iloc[0]:.2f}M")
print(f"Predicted Pipeline (PPV): ${df1['PREDICTED_PIPELINE_M'].iloc[0]:.2f}M")
print(f"Unique Clients: {df1['UNIQUE_CLIENTS'].iloc[0]}")
print(f"Total Opportunities: {df1['TOTAL_OPPORTUNITIES'].iloc[0]}")

NameError: name 'pd' is not defined

### Query 2: Pipeline by Geography

In [4]:
query2 = """
SELECT 
    GEOGRAPHY,
    SUM(OPPORTUNITY_VALUE) / 1000000.0 AS PIPELINE_M,
    SUM(QUALIFY_PLUS_AMT) / 1000000.0 AS QUALIFIED_M,
    SUM(PPV_AMT) / 1000000.0 AS FORECAST_M,
    COUNT(*) AS DEAL_COUNT,
    COUNT(DISTINCT CLIENT_NAME) AS CLIENT_COUNT
FROM 
    PROD_MQT_CONSULTING_PIPELINE
WHERE 
    SALES_STAGE NOT IN ('Won', 'Lost')
GROUP BY 
    GEOGRAPHY
ORDER BY 
    PIPELINE_M DESC;
"""

df2 = pd.read_sql_query(query2, conn)
print("🌍 PIPELINE BY GEOGRAPHY:")
print(df2)

NameError: name 'pd' is not defined

### Query 3: Sales Stage Distribution

In [5]:
query3 = """
SELECT 
    SALES_STAGE,
    COUNT(*) AS DEAL_COUNT,
    SUM(OPPORTUNITY_VALUE) / 1000000.0 AS STAGE_VALUE_M,
    AVG(OPPORTUNITY_VALUE) / 1000.0 AS AVG_DEAL_SIZE_K,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM PROD_MQT_CONSULTING_PIPELINE), 1) AS PCT_OF_DEALS
FROM 
    PROD_MQT_CONSULTING_PIPELINE
GROUP BY 
    SALES_STAGE
ORDER BY 
    CASE SALES_STAGE
        WHEN 'Won' THEN 1
        WHEN 'Closing' THEN 2
        WHEN 'Negotiate' THEN 3
        WHEN 'Propose' THEN 4
        WHEN 'Qualify' THEN 5
        ELSE 6
    END;
"""

df3 = pd.read_sql_query(query3, conn)
print("📈 SALES STAGE DISTRIBUTION:")
print(df3)

NameError: name 'pd' is not defined

### Query 4: Win Rate Analysis

In [6]:
query4 = """
SELECT 
    GEOGRAPHY,
    SUM(CASE WHEN SALES_STAGE = 'Won' THEN 1 ELSE 0 END) AS WON_DEALS,
    COUNT(*) AS TOTAL_DEALS,
    ROUND(SUM(CASE WHEN SALES_STAGE = 'Won' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS WIN_RATE_PCT,
    SUM(CASE WHEN SALES_STAGE = 'Won' THEN OPPORTUNITY_VALUE ELSE 0 END) / 1000000.0 AS WON_VALUE_M
FROM 
    PROD_MQT_CONSULTING_PIPELINE
GROUP BY 
    GEOGRAPHY
ORDER BY 
    WIN_RATE_PCT DESC;
"""

df4 = pd.read_sql_query(query4, conn)
print("🏆 WIN RATE BY GEOGRAPHY:")
print(df4)

NameError: name 'pd' is not defined

### Query 5: Forecast vs Budget Analysis

In [7]:
query5 = """
SELECT 
    p.GEOGRAPHY,
    p.MARKET,
    SUM(p.PPV_AMT) / 1000000.0 AS FORECASTED_REVENUE_M,
    b.REVENUE_BUDGET_AMT / 1000000.0 AS BUDGET_REVENUE_M,
    ROUND((SUM(p.PPV_AMT) / b.REVENUE_BUDGET_AMT) * 100, 1) AS FORECAST_TO_BUDGET_PCT,
    SUM(p.WON_AMT) / 1000000.0 AS WON_TO_DATE_M
FROM 
    PROD_MQT_CONSULTING_PIPELINE p
LEFT JOIN 
    PROD_MQT_CONSULTING_BUDGET b
    ON p.GEOGRAPHY = b.GEOGRAPHY 
    AND p.MARKET = b.MARKET
    AND p.YEAR = b.YEAR
    AND p.QUARTER = b.QUARTER
WHERE 
    b.REVENUE_BUDGET_AMT IS NOT NULL
GROUP BY 
    p.GEOGRAPHY, p.MARKET, b.REVENUE_BUDGET_AMT
ORDER BY 
    FORECAST_TO_BUDGET_PCT DESC;
"""

df5 = pd.read_sql_query(query5, conn)
print("🎯 FORECAST VS BUDGET:")
print(df5)

NameError: name 'pd' is not defined

## 3. Create Visualizations

In [8]:
# Create comprehensive dashboard
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('IBM Sales Pipeline Analytics Dashboard', fontsize=16, fontweight='bold')

# 1. Pipeline by Geography
ax1 = axes[0, 0]
df2.plot(x='GEOGRAPHY', y=['PIPELINE_M', 'QUALIFIED_M', 'FORECAST_M'], 
         kind='bar', ax=ax1, width=0.8)
ax1.set_title('Pipeline by Geography ($M)', fontweight='bold')
ax1.set_ylabel('Amount ($M)')
ax1.legend(['Total Pipeline', 'Qualified', 'Forecast'])
ax1.tick_params(axis='x', rotation=45)

# 2. Sales Stage Distribution (Pie Chart)
ax2 = axes[0, 1]
ax2.pie(df3['DEAL_COUNT'], labels=df3['SALES_STAGE'], autopct='%1.1f%%', startangle=90)
ax2.set_title('Deal Distribution by Stage', fontweight='bold')

# 3. Win Rate vs Won Value (Scatter)
ax3 = axes[0, 2]
scatter = ax3.scatter(df4['WIN_RATE_PCT'], df4['WON_VALUE_M'], 
                     s=df4['WON_VALUE_M']*100, alpha=0.6, c=['red', 'blue', 'green'])
for idx, row in df4.iterrows():
    ax3.annotate(row['GEOGRAPHY'], (row['WIN_RATE_PCT'], row['WON_VALUE_M']),
                fontsize=10, ha='center')
ax3.set_xlabel('Win Rate (%)')
ax3.set_ylabel('Won Value ($M)')
ax3.set_title('Win Rate vs Won Value', fontweight='bold')
ax3.grid(True, alpha=0.3)

# 4. Forecast vs Budget
ax4 = axes[1, 0]
x = np.arange(len(df5))
width = 0.35
ax4.bar(x - width/2, df5['FORECASTED_REVENUE_M'], width, label='Forecast', color='lightblue')
ax4.bar(x + width/2, df5['BUDGET_REVENUE_M'], width, label='Budget', color='lightcoral')
ax4.set_xlabel('Market')
ax4.set_ylabel('Amount ($M)')
ax4.set_title('Forecast vs Budget by Market', fontweight='bold')
ax4.set_xticks(x)
ax4.set_xticklabels(df5['MARKET'], rotation=45, ha='right')
ax4.legend()

# Add coverage percentage labels
for i, pct in enumerate(df5['FORECAST_TO_BUDGET_PCT']):
    ax4.text(i, max(df5['FORECASTED_REVENUE_M'].iloc[i], df5['BUDGET_REVENUE_M'].iloc[i]) + 0.05,
            f'{pct}%', ha='center', fontweight='bold', fontsize=9)

# 5. Industry Distribution
ax5 = axes[1, 1]
industry_query = """
SELECT 
    INDUSTRY,
    SUM(OPPORTUNITY_VALUE) / 1000000.0 AS PIPELINE_M,
    COUNT(*) AS DEAL_COUNT
FROM 
    PROD_MQT_CONSULTING_PIPELINE
WHERE 
    SALES_STAGE NOT IN ('Won', 'Lost')
GROUP BY 
    INDUSTRY
ORDER BY 
    PIPELINE_M DESC;
"""
df_industry = pd.read_sql_query(industry_query, conn)

# Horizontal bar chart for industries
y_pos = np.arange(len(df_industry))
ax5.barh(y_pos, df_industry['PIPELINE_M'], color='green', alpha=0.7)
ax5.set_yticks(y_pos)
ax5.set_yticklabels(df_industry['INDUSTRY'])
ax5.set_xlabel('Pipeline Value ($M)')
ax5.set_title('Pipeline by Industry', fontweight='bold')

# 6. Service Line Performance
ax6 = axes[1, 2]
service_query = """
SELECT 
    UT17_NAME,
    SUM(OPPORTUNITY_VALUE) / 1000000.0 AS PIPELINE_M,
    COUNT(*) AS DEAL_COUNT
FROM 
    PROD_MQT_CONSULTING_PIPELINE
WHERE 
    SALES_STAGE NOT IN ('Won', 'Lost')
GROUP BY 
    UT17_NAME
ORDER BY 
    PIPELINE_M DESC;
"""
df_service = pd.read_sql_query(service_query, conn)

ax6.pie(df_service['PIPELINE_M'], labels=df_service['UT17_NAME'], autopct='%1.1f%%', startangle=90)
ax6.set_title('Pipeline by Service Line', fontweight='bold')

plt.tight_layout()
plt.show()

print("✅ Dashboard created successfully!")

NameError: name 'plt' is not defined

## 4. Executive Summary Report

In [9]:
print("📋 EXECUTIVE SUMMARY REPORT")
print("=" * 50)

# Key metrics
total_pipeline = df1['TOTAL_PIPELINE_M'].iloc[0]
qualified_pipeline = df1['QUALIFIED_PIPELINE_M'].iloc[0]
predicted_revenue = df1['PREDICTED_PIPELINE_M'].iloc[0]
unique_clients = df1['UNIQUE_CLIENTS'].iloc[0]

print(f"\n🎯 KEY METRICS:")
print(f"   Total Pipeline: ${total_pipeline:.2f}M")
print(f"   Qualified Pipeline: ${qualified_pipeline:.2f}M")
print(f"   Forecasted Revenue (PPV): ${predicted_revenue:.2f}M")
print(f"   Unique Clients: {unique_clients}")

# Top performing geography
top_geo = df2.iloc[0]
print(f"\n🌟 TOP PERFORMING GEOGRAPHY:")
print(f"   {top_geo['GEOGRAPHY']}: ${top_geo['PIPELINE_M']:.2f}M pipeline")
print(f"   {top_geo['CLIENT_COUNT']} clients, {top_geo['DEAL_COUNT']} deals")

# Win rate insights
best_win_rate = df4.iloc[0]
print(f"\n🏆 WIN RATE LEADER:")
print(f"   {best_win_rate['GEOGRAPHY']}: {best_win_rate['WIN_RATE_PCT']}% win rate")
print(f"   Won Value: ${best_win_rate['WON_VALUE_M']:.2f}M")

# Budget performance
avg_coverage = df5['FORECAST_TO_BUDGET_PCT'].mean()
print(f"\n📊 BUDGET PERFORMANCE:")
print(f"   Average Forecast Coverage: {avg_coverage:.1f}%")
if avg_coverage >= 100:
    print(f"   ✅ On track to meet targets")
elif avg_coverage >= 80:
    print(f"   ⚠️ Slight gap to targets - need {100-avg_coverage:.1f}% more pipeline")
else:
    print(f"   🚨 Significant gap to targets - need {100-avg_coverage:.1f}% more pipeline")

# Action items
print(f"\n🎯 RECOMMENDED ACTIONS:")
print(f"   1. Focus on {df2.iloc[-1]['GEOGRAPHY']} geography (lowest pipeline)")
print(f"   2. Accelerate deals in {df3[df3['SALES_STAGE'].isin(['Qualify', 'Propose'])]['DEAL_COUNT'].sum()} early-stage opportunities")
print(f"   3. Push {df3[df3['SALES_STAGE'] == 'Closing']['DEAL_COUNT'].iloc[0]} closing deals to Won")
print(f"   4. Expand client base (currently {unique_clients} unique clients)")

print("\n" + "=" * 50)

📋 EXECUTIVE SUMMARY REPORT


NameError: name 'df1' is not defined

## 5. Additional Analysis Queries

### Query: Which deals need follow-up this week?

In [10]:
followup_query = """
SELECT 
    CLIENT_NAME,
    OPPORTUNITY_VALUE / 1000.0 AS VALUE_K,
    SALES_STAGE,
    GEOGRAPHY,
    MARKET,
    CASE 
        WHEN SALES_STAGE = 'Negotiate' AND CALL_AMT = 0 
        THEN 'Critical - In Negotiate without commitment'
        WHEN SALES_STAGE = 'Propose' AND UPSIDE_AMT > 0 
        THEN 'High - Proposal needs push'
        WHEN SALES_STAGE = 'Qualify' AND OPPORTUNITY_VALUE > 500000 
        THEN 'High - Large deal in early stage'
        ELSE 'Standard follow-up'
    END AS FOLLOW_UP_PRIORITY,
    CASE 
        WHEN SALES_STAGE = 'Negotiate' 
        THEN 'Schedule executive meeting to close'
        WHEN SALES_STAGE = 'Propose' 
        THEN 'Address proposal questions and objections'
        WHEN SALES_STAGE = 'Qualify' 
        THEN 'Confirm budget and decision criteria'
        ELSE 'Standard check-in'
    END AS RECOMMENDED_ACTION
FROM 
    PROD_MQT_CONSULTING_PIPELINE
WHERE 
    SALES_STAGE NOT IN ('Won', 'Lost', 'Closing')
ORDER BY 
    CASE 
        WHEN SALES_STAGE = 'Negotiate' AND CALL_AMT = 0 THEN 1
        WHEN SALES_STAGE = 'Propose' AND UPSIDE_AMT > 0 THEN 2
        WHEN SALES_STAGE = 'Qualify' AND OPPORTUNITY_VALUE > 500000 THEN 3
        ELSE 4
    END,
    OPPORTUNITY_VALUE DESC;
"""

df_followup = pd.read_sql_query(followup_query, conn)
print("📞 DEALS REQUIRING FOLLOW-UP:")
print(df_followup)

NameError: name 'pd' is not defined

### Query: Pipeline Health Score by Market

In [11]:
health_query = """
WITH pipeline_health AS (
    SELECT 
        MARKET,
        SUM(CASE WHEN SALES_STAGE = 'Closing' THEN OPPORTUNITY_VALUE ELSE 0 END) / 
            NULLIF(SUM(QUALIFY_PLUS_AMT), 0) * 100 AS CLOSING_PCT,
        SUM(PPV_AMT) / NULLIF(SUM(QUALIFY_PLUS_AMT), 0) * 100 AS PPV_RATIO,
        COUNT(DISTINCT CLIENT_NAME) AS CLIENT_DIVERSITY,
        AVG(OPPORTUNITY_VALUE) / 1000000.0 AS AVG_DEAL_SIZE_M
    FROM 
        PROD_MQT_CONSULTING_PIPELINE
    WHERE 
        SALES_STAGE NOT IN ('Won', 'Lost')
    GROUP BY 
        MARKET
    HAVING 
        SUM(QUALIFY_PLUS_AMT) > 0
)
SELECT 
    MARKET,
    ROUND(CLOSING_PCT, 1) AS CLOSING_PCT,
    ROUND(PPV_RATIO, 1) AS PPV_RATIO,
    CLIENT_DIVERSITY,
    ROUND(AVG_DEAL_SIZE_M, 2) AS AVG_DEAL_SIZE_M,
    ROUND((CLOSING_PCT * 0.4 + PPV_RATIO * 0.3 + 
     CASE WHEN CLIENT_DIVERSITY > 5 THEN 20 ELSE CLIENT_DIVERSITY * 4 END +
     CASE WHEN AVG_DEAL_SIZE_M > 0.5 THEN 10 ELSE AVG_DEAL_SIZE_M * 20 END), 1) AS HEALTH_SCORE
FROM 
    pipeline_health
ORDER BY 
    HEALTH_SCORE DESC;
"""

df_health = pd.read_sql_query(health_query, conn)
print("💊 PIPELINE HEALTH SCORES:")
print(df_health)

# Close database connection
conn.close()
print("\n✅ Demo completed successfully!")

NameError: name 'pd' is not defined